# Background Information

In this notebook I will set up a basic flow for uploading and dowloading files. Formating datasets and ultimately running cytetype.

In [1]:
# Importing Require Libraries
import cytetype
import scanpy as sc
import session_info
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
import json

load_dotenv()

True

In [2]:
# Session Info
session_info.show()

/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [3]:
# Importing AnnData file
# File Path
#file_path_Simone = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Simone_et_al/7d821d98-5b42-4480-8173-641c1b37b237.h5ad")
#file_path_Granja = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Granja_et_al/scRNA-Hematopoiesis-Granja-2019.h5ad")
#file_path_Jardine = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Jardine_et_al/fig1b_fbm_scaled_gex_updated_dr_20210104.h5ad")
#file_path_Hiemlich = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Heimlich_et_al/5e3e4027-0675-4d5f-b223-bb03a17ade71.h5ad")
#file_path_Roy = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Roy_et_al/GSE155259_Roy_et_al.h5ad")
file_path_Zhang = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Zhang_et_al/adata_combined_rna_adt_annotated-titration.h5ad")

# AnnData Read In
#adata = sc.read_h5ad(file_path_Simone)
#adata = sc.read_h5ad(file_path_Hiemlich)
#adata = sc.read_h5ad(file_path_Roy)
adata = sc.read_h5ad(file_path_Zhang)

In [4]:
adata.X = adata.raw.X.copy()

AttributeError: 'NoneType' object has no attribute 'X'

In [10]:
# Normalize (10,000 Counts per Cell)
sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)

# Log1p transform
sc.pp.log1p(adata)

In [8]:
# Explore log1p Error (It is due to NaNs)
print("Min:", np.min(adata.X))
print("Any NaNs in sparse data?", np.isnan(adata.X.data).any())
print("Any Infs in sparse data?", np.isinf(adata.X.data).any())

Min: 0.0
Any NaNs in sparse data? False
Any Infs in sparse data? False


In [4]:
# Zhang
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='Level 2 RNA', method='t-test', key_added = 'Rank_Genes_cell_type')  # try 'wilcoxon' if accurate

/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:456: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:458: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_gene

In [12]:
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='cell_type', method='t-test', key_added = 'Rank_Genes_cell_type')  # try 'wilcoxon' if accurate

/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:456: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:458: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_gene

In [6]:
# Jardine
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='broad_fig1_cell.labels', method='t-test', key_added = 'Rank_Genes_cell_type', inplace = True)  # try 'wilcoxon' if accurate

/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:479: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (


In [6]:
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='BioClassification', method='t-test', key_added = 'Rank_Genes_cell_type')  # try 'wilcoxon' if accurate

/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:456: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:458: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_gene

In [7]:
annotator = cytetype.CyteType(adata, group_key = 'cell_type', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'gene_name')

KeyError: "Cell group key 'cell_type' not found in `adata.obs`."

In [7]:
adata.var["gene_symbols"] = adata.var.index
annotator = cytetype.CyteType(adata, group_key = 'cell_type', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'gene_symbols')

Calculating expression percentages.
Extracting marker genes.
Data preparation completed. Ready for submitting jobs.


In [5]:
# Zhang
adata.var["gene_symbols"] = adata.var.index
annotator = cytetype.CyteType(adata, group_key = 'Level 2 RNA', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'gene_symbols')

Calculating expression percentages.
Extracting marker genes.
Data preparation completed. Ready for submitting jobs.


In [14]:
#Roy
annotator = cytetype.CyteType(adata, group_key = 'cell_type', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'name')

Calculating expression percentages.
Extracting marker genes.
Data preparation completed. Ready for submitting jobs.


In [12]:
adata.var["gene_symbols"] = adata.var.index
annotator = cytetype.CyteType(adata, group_key = 'broad_fig1_cell.labels', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'gene_symbols')

Calculating expression percentages.
Extracting marker genes.
Data preparation completed. Ready for submitting jobs.


In [11]:
annotator = cytetype.CyteType(adata, group_key = 'BioClassification', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'name')

Calculating expression percentages.
Extracting marker genes.
Data preparation completed. Ready for submitting jobs.


In [57]:
# Run Annotation with custom model
adata = annotator.run(
    study_context = "Healthy human fetal bone marrow cell, analysed 10x genomics single cell RNA-seq kits",
    model_config = [{
        'provider': 'openai',
        'name': 'qwen/qwen3-235b-a22b',
        'apiKey': os.environ.get('OPENROUTER_API_KEY'),
        'baseUrl': "https://openrouter.ai/api/v1",
    }],
    auth_token = os.environ.get('NYGEN_API_KEY'),
    timeout_seconds=3000,
    run_config={
        'concurrentClusters': 5,        # Default: 3, Range: 2-10
        'maxAnnotationRevisions': 2,    # Default: 2, Range: 1-5
        'maxLLMRequests': 250           # Default: 500, Range: 50-2000
    },
)

Waiting for results for job ID: 320eac1b-2e03-40c3-844e-336a58ab3db7
View the automatically updating visualization report at: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/report/320eac1b-2e03-40c3-844e-336a58ab3db7
250605:1154:22 |INFO| [STARTING WORKFLOWS]: Job ID: 320eac1b-2e03-40c3-844e-336a58ab3db7
250605:1154:22 |INFO| [WORKFLOW STEP 1/3]: Generating context summary
250605:1155:41 |INFO| [WORKFLOW STEP 2/3]: Running cluster contextualization, annotation, review, and ontology assignment
250605:1155:43 |INFO| [Cluster: 1] Generating cluster-specific context...
250605:1155:46 |INFO| [Cluster: 10] Generating cluster-specific context...
250605:1155:51 |INFO| [Cluster: 2] Generating cluster-specific context...
250605:1155:52 |INFO| [Cluster: 3] Generating cluster-specific context...
250605:1155:53 |INFO| [Cluster: 4] Generating cluster-specific context...
250605:1156:21 |INFO| [Cluster: 3] Annotating cluster...
250605:1156:23 |INFO| [Cluster: 2] Annotating cluste

In [13]:
# Run annotation with custom model
adata = annotator.run(
    study_context = "Human adult mixed-phenotype acute leukemias from peripheral blood and bone marrow, analysed 10x genomics single cell RNA-seq kits",
    model_config = [{
        'provider': 'openai',
        'name': 'qwen/qwen3-235b-a22b',
        'apiKey': os.environ.get('OPENROUTER_API_KEY'),
        'baseUrl': "https://openrouter.ai/api/v1",
    }],
    auth_token = os.environ.get('NYGEN_API_KEY'),
    timeout_seconds=3000,
    run_config={
        'concurrentClusters': 5,        # Default: 3, Range: 2-10
        'maxAnnotationRevisions': 2,    # Default: 2, Range: 1-5
        'maxLLMRequests': 250           # Default: 500, Range: 50-2000
    },
)

Waiting for results for job ID: 54d0c1f1-d922-4c24-bae0-e43930fbee49
View the automatically updating visualization report at: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/report/54d0c1f1-d922-4c24-bae0-e43930fbee49
Could not fetch logs for job 54d0c1f1-d922-4c24-bae0-e43930fbee49: 404 Client Error: Not Found for url: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/logs/54d0c1f1-d922-4c24-bae0-e43930fbee49
250605:1003:46 |INFO| [STARTING WORKFLOWS]: Job ID: 54d0c1f1-d922-4c24-bae0-e43930fbee49
250605:1003:46 |INFO| [WORKFLOW STEP 1/3]: Generating context summary
250605:1004:42 |INFO| [WORKFLOW STEP 2/3]: Running cluster contextualization, annotation, review, and ontology assignment
250605:1004:46 |INFO| [Cluster: 1] Generating cluster-specific context...
250605:1004:50 |INFO| [Cluster: 10] Generating cluster-specific context...
250605:1004:55 |INFO| [Cluster: 2] Generating cluster-specific context...
250605:1004:59 |INFO| [Cluster: 3] Generat

In [23]:
# Heimlich
# Run annotation with custom model
adata = annotator.run(
    study_context = "Human adult with clonal hematopoiesis or healthy peripheral blood, analysed 10x genomics single cell RNA-seq kits",
    model_config = [{
        'provider': 'openai',
        'name': 'meta-llama/llama-4-maverick',
        'apiKey': os.environ.get('OPENROUTER_API_KEY'),
        'baseUrl': "https://openrouter.ai/api/v1",
    }],
    auth_token = os.environ.get('NYGEN_API_KEY'),
    timeout_seconds=3000,
    run_config={
        'concurrentClusters': 5,        # Default: 3, Range: 2-10
        'maxAnnotationRevisions': 2,    # Default: 2, Range: 1-5
        'maxLLMRequests': 225           # Default: 500, Range: 50-2000
    },
)

Waiting for results for job ID: 6248f067-9da2-444c-8ee5-526186ab2ef0
View the automatically updating visualization report at: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/report/6248f067-9da2-444c-8ee5-526186ab2ef0
250610:1046:11 |INFO| [STARTING WORKFLOWS]: Job ID: 6248f067-9da2-444c-8ee5-526186ab2ef0
250610:1046:11 |INFO| [WORKFLOW STEP 1/3]: Generating context summary
250610:1046:18 |INFO| [WORKFLOW STEP 2/3]: Running complete annotation pipeline for each cluster
250610:1046:19 |INFO| [Cluster: 1] Generating cluster-specific context...
250610:1046:19 |INFO| [Cluster: 2] Generating cluster-specific context...
250610:1046:20 |INFO| [Cluster: 3] Generating cluster-specific context...
250610:1046:21 |INFO| [Cluster: 4] Generating cluster-specific context...
250610:1046:22 |INFO| [Cluster: 5] Generating cluster-specific context...
250610:1046:24 |INFO| [Cluster: 1] Attempt 1: Running complete annotation pipeline...
250610:1046:24 |INFO| [Cluster: 4] Attempt 1: Run

In [6]:
# Zhang
# Run annotation with custom model
adata = annotator.run(
    study_context = "Healthy adult human bone marrow, analysed using 10x genomics single cell RNA-seq kits",
    model_config = [{
        'provider': 'openai',
        'name': 'qwen/qwen3-235b-a22b',
        'apiKey': os.environ.get('OPENROUTER_API_KEY'),
        'baseUrl': "https://openrouter.ai/api/v1",
    }],
    auth_token = os.environ.get('NYGEN_API_KEY'),
    timeout_seconds=3900,
    run_config={
        'concurrentClusters': 5,        # Default: 3, Range: 2-10
        'maxAnnotationRevisions': 2,    # Default: 2, Range: 1-5
        'maxLLMRequests': 900           # Default: 500, Range: 50-2000
    },
)

Waiting for results for job ID: ba82f2af-cf37-4548-ac39-be56cc2441b5
View the automatically updating visualization report at: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/report/ba82f2af-cf37-4548-ac39-be56cc2441b5
250613:0712:02 |INFO| [STARTING WORKFLOWS]: Job ID: ba82f2af-cf37-4548-ac39-be56cc2441b5
250613:0712:02 |INFO| [WORKFLOW STEP 1/3]: Generating context summary
250613:0713:31 |INFO| [WORKFLOW STEP 2/3]: Running complete annotation pipeline for each cluster
250613:0713:32 |INFO| [Cluster: 1] Generating cluster-specific context...
250613:0713:33 |INFO| [Cluster: 10] Generating cluster-specific context...
250613:0713:35 |INFO| [Cluster: 11] Generating cluster-specific context...
250613:0713:36 |INFO| [Cluster: 12] Generating cluster-specific context...
250613:0713:36 |INFO| [Cluster: 13] Generating cluster-specific context...
250613:0714:27 |INFO| [Cluster: 10] Attempt 1: Running complete annotation pipeline...
250613:0714:30 |INFO| [Cluster: 13] Attempt 

CyteTypeJobError: Server error: job failed

In [43]:
# Roy
# Run annotation with custom model
adata = annotator.run(
    study_context = "Healthy human hematopoietic tissues from the first trimester (fetal liver), paired second trimester (fetal bone marrow and fetal liver from the same fetuses), pediatric bone marrow, and adult bone marrow, analysed using 10x genomics single cell RNA-seq kits",
    model_config = [{
        'provider': 'openai',
        'name': 'openai/gpt-4.1',
        'apiKey': os.environ.get('OPENROUTER_API_KEY'),
        'baseUrl': "https://openrouter.ai/api/v1",
    }],
    auth_token = os.environ.get('NYGEN_API_KEY'),
    timeout_seconds=3000,
    run_config={
        'concurrentClusters': 5,        # Default: 3, Range: 2-10
        'maxAnnotationRevisions': 2,    # Default: 2, Range: 1-5
        'maxLLMRequests': 525           # Default: 500, Range: 50-2000
    },
)

Waiting for results for job ID: 9b2e9900-7410-4380-84ac-ccefcdbe2498
View the automatically updating visualization report at: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/report/9b2e9900-7410-4380-84ac-ccefcdbe2498
250610:1300:28 |INFO| [STARTING WORKFLOWS]: Job ID: 9b2e9900-7410-4380-84ac-ccefcdbe2498
250610:1300:28 |INFO| [WORKFLOW STEP 1/3]: Generating context summary
250610:1300:51 |INFO| [WORKFLOW STEP 2/3]: Running complete annotation pipeline for each cluster
250610:1300:52 |INFO| [Cluster: 1] Generating cluster-specific context...
250610:1300:52 |INFO| [Cluster: 10] Generating cluster-specific context...
250610:1300:53 |INFO| [Cluster: 11] Generating cluster-specific context...
250610:1300:55 |INFO| [Cluster: 12] Generating cluster-specific context...
250610:1300:56 |INFO| [Cluster: 13] Generating cluster-specific context...
250610:1301:02 |INFO| [Cluster: 10] Attempt 1: Running complete annotation pipeline...
250610:1301:02 |INFO| [Cluster: 11] Attempt 

In [ ]:
Author_vs_LLM = pd.crosstab(adata.obs['Level 2 RNA'], adata.obs['cytetype_annotation_Level 2 RNA']).T.idxmax()

In [ ]:
output_path = "/Users/alexantill/Göran_Karlsson_Lab/benchLLM/Zhang_et_al/Qwen3.json"

with open(output_path, "w") as f:
    json.dump(adata.uns['cytetype_results']['result'], f, indent=2)

In [ ]:
Author_vs_LLM.reset_index().to_csv("/Users/alexantill/Göran_Karlsson_Lab/benchLLM/Zhang_et_al/author_vs_Qwen3.csv", index=False)

In [ ]:
models = [
    "google/gemini-2.5-pro-preview",
    "anthropic/claude-3.7-sonnet:thinking",
    "openai/o3-mini",
    "openai/o4-mini",
    "x-ai/grok-3-beta",
    "meta-llama/llama-4-maverick",
    "anthropic/claude-3.7-sonnet",
    "openai/gpt-4.1-mini",
    "qwen/qwen3-235b-a22b",
    "google/gemini-2.0-flash-001",
    "openai/gpt-4.1"
]

results = {}

for model_full in models:
    model_name = model_full.split("/")[-1]
    adata_result = annotator.run(
        study_context="Healthy adult human peripheral blood, analysed on a variety of commerical single cell RNA-seq kits",
        model_config=[{
            'provider': 'openai',  # update this if provider changes!
            'name': model_full,
            'apiKey': os.environ.get('OPENROUTER_API_KEY'),
            'baseUrl': "https://openrouter.ai/api/v1",
        }],
        auth_token=os.environ.get('NYGEN_API_KEY'),
        timeout_seconds=1500,
    )
    results[f'adata_{model_name}'] = adata_result

    # Create crosstab and get idxmax
    cross_tab = pd.crosstab(
        adata_result.obs.cell_type,
        adata_result.obs.cytetype_cell_type
    ).T.idxmax()

    cross_tab_df = cross_tab.reset_index()
    
    # Save to CSV with model-specific filename
    output_path = f"/Users/alexantill/Göran_Karlsson_Lab/benchLLM/Simone_et_al/author_vs_{model_name}.csv"
    cross_tab_df.to_csv(output_path, index=False)
    print(f"Saved crosstab for {model_name} to {output_path}")
